In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

2023-10-07 13:23:31.864324: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-07 13:23:31.866656: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-07 13:23:31.914707: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-07 13:23:31.915721: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-07 13:23:32.729125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

<IPython.core.display.Javascript object>

# Functions and definitions

### Data preprocessing

In [3]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [5]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [6]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

## Sequence helper

In [7]:
def generate_sequences(dataset, timesteps):
    dataset["x_train"], dataset["y_train"] = split_sequences(
        pd.concat(
            [
                pd.DataFrame(dataset["x_train"], columns=x.columns),
                dataset["y_train"],
            ],
            axis=1,
        ).values,
        timesteps,
    )

    dataset["x_test"], dataset["y_test"] = split_sequences(
        pd.concat(
            [
                pd.DataFrame(dataset["x_test"], columns=x.columns),
                dataset["y_test"],
            ],
            axis=1,
        ).values,
        timesteps,
    )
    return dataset

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [8]:
def custom_cross_validate(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    estimator_params=None,
    imputer_params=None,
):
    results = []
    scores = []

    for train_index, test_index in cv.split(x):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        scores.append(score)

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]
    results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [9]:
class LSTM(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM(units=16, activation="relu"))
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            # optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [10]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [11]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [12]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Dataframe structure to save the results

In [13]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_iv",
    "Features": "Chemical + Mineralogical + Physical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "LSTM",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [14]:
df = pd.read_csv("../../../../../data/processed/partner_iv/cem_c.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

In [15]:
df_copy = df.copy()

<IPython.core.display.Javascript object>

<h2>1. Dataset: df_copy</h2> <br>In this dataset all features are used.

In [16]:
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [17]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# 1. Long Short Term Memory - LSTM

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (410, 37)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 41 samples each
    2. 90% train (369 samples each fold)
    3. 10% test (41 samples each fold)
<b>Total:</b> 100 models<br>

In [18]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"], axis=1)
    y = df_copy["CS28"]
    scores = custom_cross_validate(
        LSTM,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        estimator_params=None,
        imputer_params={"strategy": "median"},
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



2023-10-07 13:23:35.430955: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-07 13:23:35.431015: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2023-10-07 13:23:35.431024: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2023-10-07 13:23:35.431220: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.125.6
2023-10-07 13:23:35.431253: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.125.6
2023-10-07 13:23:35.431260: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.125.6


TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.919 (0.340)
MAE: 2.160 (0.259)
MAPE: 0.053 (0.006)
R2: -0.125 (0.243)


******
[TEST]
******
RMSE: 6.413 (0.779)
MAE: 5.021 (0.608)
MAPE: 0.123 (0.015)
R2: -4.621 (1.578)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.472 (0.182)
MAE: 1.194 (0.147)
MAPE: 0.029 (0.004)
R2: 0.710 (0.076)


******
[TEST]
******
RMSE: 5.427 (1.415)
MAE: 4.265 (1.025)
MAPE: 0.105 (0.025)
R2: -3.148 (2.196)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.364 (0.154)
MAE: 1.094 (0.127)
MAPE: 0.027 (0.003)
R2: 0.753 (0.057)


******
[TEST]
******
RMSE: 4.081 (1.022)
MAE: 3.181 (0.818)
MAPE: 0.079 (0.021)
R2: -1.234 (1.175)


Minutes Elapsed:  22.45665067434311


<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (410, 37)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 82 samples
    2. 50% train (41 samples each fold)
    3. 50% test (41 samples each fold)
<b>Total:</b> 5 models<br>

In [19]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 8.997 (1.317)
MAE: 6.785 (1.154)
MAPE: 0.168 (0.029)
R2: -14.486 (6.649)


******
[TEST]
******
RMSE: 18.126 (4.669)
MAE: 16.049 (4.551)
MAPE: 0.395 (0.118)
R2: -163.792 (99.075)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.800 (0.364)
MAE: 1.479 (0.338)
MAPE: 0.037 (0.009)
R2: 0.315 (0.309)


******
[TEST]
******
RMSE: 20.680 (7.096)
MAE: 19.600 (7.513)
MAPE: 0.482 (0.203)
R2: -275.214 (279.717)


Minutes Elapsed:  3.5339183529218037


<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (410, 37)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 40, 77, 114, 152, 189, 226, 263, 301, 338 samples each fold
    2. Test: 37 samples each fold
<b>Total:</b> 10 models<br>

In [20]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 5.054 (2.346)
MAE: 3.792 (1.823)
MAPE: 0.093 (0.046)
R2: -3.370 (4.424)


******
[TEST]
******
RMSE: 13.120 (4.155)
MAE: 10.867 (3.838)
MAPE: 0.268 (0.097)
R2: -33.562 (22.402)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.675 (0.310)
MAE: 1.347 (0.258)
MAPE: 0.033 (0.007)
R2: 0.583 (0.225)


******
[TEST]
******
RMSE: 15.097 (5.649)
MAE: 12.559 (4.617)
MAPE: 0.311 (0.119)
R2: -46.079 (34.474)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.689 (0.342)
MAE: 1.390 (0.288)
MAPE: 0.034 (0.007)
R2: 0.584 (0.215)


******
[TEST]
******
RMSE: 14.492 (7.527)
MAE: 12.344 (6.733)
MAPE: 0.304 (0.162)
R2: -47.335 (54.441)


Minutes Elapsed:  14.905287086963654


<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (410, 37)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 328
    2. Test: 82
<b>Total:</b> 1 model<br>

In [21]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        # Data Splitting
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)

        # Sequence Splitting
        data_train = pd.concat(
            [
                pd.DataFrame(x_train, columns=x.columns),
                y_train.reset_index(drop=True),
            ],
            axis=1,
        ).values
        data_test = pd.concat(
            [
                pd.DataFrame(x_test, columns=x.columns),
                y_test.reset_index(drop=True),
            ],
            axis=1,
        ).values

        x_train, y_train = split_sequences(data_train, timesteps)
        x_test, y_test = split_sequences(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", LSTM())])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.894 (0.377)
MAE: 2.076 (0.187)
MAPE: 0.050 (0.004)
R2: -0.001 (0.266)


******
[TEST]
******
RMSE: 9.619 (0.739)
MAE: 7.985 (0.731)
MAPE: 0.197 (0.018)
R2: -24.039 (3.724)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.472 (0.226)
MAE: 1.185 (0.184)
MAPE: 0.029 (0.005)
R2: 0.738 (0.082)


******
[TEST]
******
RMSE: 6.844 (1.312)
MAE: 5.485 (1.157)
MAPE: 0.138 (0.029)
R2: -12.825 (5.374)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.329 (0.256)
MAE: 1.090 (0.214)
MAPE: 0.027 (0.005)
R2: 0.786 (0.076)


******
[TEST]
******
RMSE: 6.632 (1.474)
MAE: 5.699 (1.474)
MAPE: 0.144 (0.037)
R2: -12.532 (5.622)


Minutes Elapsed:  4.3744322776794435


<IPython.core.display.Javascript object>

In [22]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)


Features Model            Cross Validation  \
                                                                            
0   Chemical + Mineralogical + Physical  LSTM  Blocking Time Series Split   
1   Chemical + Mineralogical + Physical  LSTM  Blocking Time Series Split   
2   Chemical + Mineralogical + Physical  LSTM           Out of time Split   
3   Chemical + Mineralogical + Physical  LSTM           Out of time Split   
4   Chemical + Mineralogical + Physical  LSTM           Out of time Split   
5   Chemical + Mineralogical + Physical  LSTM              Repeated KFold   
6   Chemical + Mineralogical + Physical  LSTM              Repeated KFold   
7   Chemical + Mineralogical + Physical  LSTM              Repeated KFold   
8   Chemical + Mineralogical + Physical  LSTM           Time Series Split   
9   Chemical + Mineralogical + Physical  LSTM           Time Series Split   
10  Chemical + Mineralogical + Physical  LSTM           Time Series Split   

   Timesteps  RMSE Test             MAE Test           MAPE Test            \
                   mean       std       mean       std      mean       std   
0          1  18.125619  4.669341  16.048638  4.551241  0.394928  0.118055   
1          7  20.680409  7.095974  19.600447  7.513290  0.481734  0.203079   
2          1   9.618615  0.738705   7.985490  0.730881  0.197097  0.017754   
3          7   6.844483  1.312042   5.485054  1.157015  0.138008  0.028625   
4         14   6.632290  1.474216   5.698967  1.473617  0.144129  0.037066   
5          1   6.413111  0.779073   5.020891  0.607723  0.123329  0.014996   
6          7   5.427063  1.414905   4.265082  1.025429  0.105035  0.025290   
7         14   4.081417  1.022498   3.181162  0.818407  0.078831  0.020909   
8          1  13.120206  4.155279  10.867202  3.838325  0.268068  0.097407   
9          7  15.096602  5.648660  12.559446  4.617014  0.310560  0.118802   
10        14  14.492335  7.527400  12.343942  6.732641  0.303642  0.161517   

       R2 Test              
          mean         std  
0  -163.791754   99.074610  
1  -275.214143  279.717452  
2   -24.039207    3.723641  
3   -12.824916    5.374098  
4   -12.532480    5.621564  
5    -4.621484    1.578040  
6    -3.147806    2.196392  
7    -1.233851    1.175326  
8   -33.562074   22.401773  
9   -46.079474   34.473604  
10  -47.334873   54.440843

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [23]:
path = "../../../../../reports/results/local_models/partner_iv/cem_c/full/"
filename = "lstm_results_full_1.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [24]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_iv/cem_c/grouped/"
filename = "lstm_results_grouped_1.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>